# EDA and Feature Engineering

In [1]:
# Data Visualization
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

# Config Reader
import configparser

# Database Connection
import firebase_admin
from firebase_admin import credentials, firestore

In [2]:
# Read config.ini file
config = configparser.ConfigParser()
config.read('./auth/config.ini')

# Get Google Firebase Auth
GCP_AUTH_PATH = config.get('firebase', 'GCP_AUTH_PATH')
cred = credentials.Certificate(GCP_AUTH_PATH)
app = firebase_admin.initialize_app(cred)

# Instantiate connection to database
db = firestore.client()

In [3]:
# Create collection references
reps_ref = db.collection("reps")
edu_ref = db.collection("edu")
votes_ref = db.collection("votes")

In [4]:
# Pull educational and representative data from database
degrees = pd.DataFrame([ doc.to_dict() for doc in edu_ref.get() ])
reps = pd.DataFrame([ doc.to_dict() for doc in reps_ref.get() ])

In [5]:
# Clean degree strings (ex. 'J.D.' -> 'JD')
degrees['degree'] = degrees['degree'].map(lambda x: ''.join(x.split('.')))

In [6]:
# Dictionary to bin degrees
cc_dict = {
    'Associates': ['AAS', 'AS', 'AA'],
    'Bachelors': ['BS', 'BA', 'SB', 'AB', 'BDiv', 'BBA', 'BEng', 'BM', 'ALB', 'BSN', 'BGS', 'BPA', 'BSBA', 'LLB'],
    'High School': ['HS'],
    'JD': ['JD'],
    'Masters - General': ['MA', 'MS', 'SM', 'MSc', 'MFA', 'MAcc'],
    'Masters - Public': ['MIA', 'MPA', 'MUP', 'MPP', 'MSW', 'MSS', 'MPH', 'MHS'],
    'Masters - Education': ['MEd', 'SYC'],
    'Masters - Law': ['LLM'],
    'Masters - Theology': ['MDiv', 'ThM'],
    'MBA': ['MBA', 'MSEM'],
    'PHD': ['PhD'],
    'Veterinary': ['DVM'],
    'Dental': ['DDS', 'DMD'],
    'MD': ['MD', 'DPM'],
    'PHD - Education': ['EdD'],
    'PHD - Theology': ['DMin'],
    'PHD - Public': ['DPA'],
    'Nursing': ['MSN', 'GrDip'],
}

In [7]:
# Bin degrees
x = degrees['degree']
cond_list = []
choice_list = []
for k, vs in cc_dict.items():
    for v in vs:
        cond_list.append(x == v)
        choice_list.append(k)

degrees['degree_group'] = np.select(cond_list, choice_list)

In [8]:
# Data groupby
institutions = degrees.groupby('_id')['institution'].apply(list)
deg_groups = degrees.groupby('_id')['degree_group'].apply(list)
degs = degrees.groupby('_id')['degree'].apply(list)

In [9]:
# Create merged DataFrame
df = reps[['_id', 'current_party', 'state', 'first_name', 'middle_name', 'last_name', 'dob', 'gender', 'congresses']]
df = df.merge(institutions, how='left', on='_id')
df = df.merge(deg_groups, how='left', on='_id')
df = df.merge(degs, how='left', on='_id')

# Replace null values
df['middle_name'] = np.where(df['middle_name'].isna(), '', df['middle_name'])

In [10]:
def clean_congresses(cong):
    return set([ int(c) for c in cong ])

In [11]:
# Add 'in_office' column
df['in_office'] = df['congresses'].map(clean_congresses).map(set([117]).issubset)

In [12]:
# Check results
print('Members in office:', df['in_office'].sum()) # 435 voting members + 6 non-voting members
print('Number of states:', len(df['state'].unique())) # 50 states + 6 at-large districts

Members in office: 441
Number of states: 56


In [13]:
class QueryData():
    '''
    Query DataFrame Object, allowing list object queries
    
    Parameters
    ----------
    df - Pandas DataFrame
    '''
    
    def __init__(self, df):
        self.data = df.reset_index(drop=True)
        
    def query_pipeline(self, pipeline=[]):
        '''
        Query data with pipeline
        
        Parameters
        ----------
        pipeline - list of comparison tuples (field, value)
        
        Returns
        -------
        DataFrame with query results
        '''
        
        result = self.data
        mask = None
        for c, v in pipeline:
            if type(self.data[c][0]) == list:
                if type(v) != list:
                    v = [v]
                mask = result[c].map(set(v).issubset)
                result = result[mask].reset_index(drop=True)
            else:
                mask = result[c].map(lambda x: x==v)
                result = result[mask].reset_index(drop=True)

        return result
    
    def get_col_vals(self, columns=[], sort=False):
        '''
        Get list of possible values for query
        
        Parameters
        ----------
        columns - list of query columns
        sort - bool to return sorted lists
        
        Returns
        -------
        Dictionary of field and list of values
        '''
        
        values = {}
        for c in columns:
            if type(self.data[c][0]) == list:
                vals = []
                for val in self.data[c]:
                    for v in val:
                        if v not in vals:
                            vals.append(v)
                if sort:
                    vals = sorted(vals)
                values[c] = vals
                
            else:
                vals = list(self.data[c].unique())
                if sort:
                    vals = sorted(vals)
                values[c] = vals
                
        return values

In [14]:
query = QueryData(df)

In [15]:
query.get_col_vals(['degree_group', 'in_office', 'gender'], sort=True)

{'degree_group': ['Associates',
  'Bachelors',
  'Dental',
  'High School',
  'JD',
  'MBA',
  'MD',
  'Masters - Education',
  'Masters - General',
  'Masters - Law',
  'Masters - Public',
  'Masters - Theology',
  'Nursing',
  'PHD',
  'PHD - Education',
  'PHD - Public',
  'PHD - Theology',
  'Veterinary'],
 'in_office': [False, True],
 'gender': ['F', 'M']}

In [16]:
query.query_pipeline([('degree_group', 'Dental'), ('gender', 'M')])

,_id,current_party,state,first_name,middle_name,last_name,dob,gender,congresses,institution,degree_group,degree,in_office
0,B001291,R,TX,Brian,,Babin,1948-03-23 00:00:00+00:00,M,"[117, 116, 115, 114]","[University of Texas Health Science Center, La...","[Dental, Bachelors]","[DDS, BS]",True
1,F000465,R,GA,A.,Drew,Ferguson,1966-11-15 00:00:00+00:00,M,"[117, 116, 115]","[University of Georgia, Augusta University]","[Bachelors, Dental]","[BS, DMD]",True
2,G000565,R,AZ,Paul,,Gosar,1958-11-22 00:00:00+00:00,M,"[117, 116, 115, 114, 113, 112]","[Creighton University, Creighton University]","[Dental, Bachelors]","[DDS, BS]",True
3,S001148,R,ID,Mike,,Simpson,1950-09-08 00:00:00+00:00,M,"[117, 116, 115, 114, 113, 112, 111, 110, 109, ...","[Washington University, Utah State University]","[Dental, Bachelors]","[DMD, BS]",True
4,V000133,R,NJ,Jefferson,,Van Drew,1953-02-23 00:00:00+00:00,M,"[117, 116, 116]","[Rutgers University, New Brunswick, Fairleigh ...","[Bachelors, Dental]","[BS, DMD]",True


In [185]:
def plot_stack(df, col_1, col_2, sort_by=None, by_pct=False):
    new_df = df.loc[df['in_office'] == True]
    opt_1 = new_df[col_1].unique()
    opt_2 = new_df[col_2].unique()
    figures = []
    if sort_by:
        sort_tups = []
    for i in opt_1:
        x = opt_2
        raw_y = []
        if by_pct:
            pct_y = []
        for j in opt_2:
            data = len(new_df.loc[(new_df[col_1] == i) & (new_df[col_2] == j)])
            raw_y.append(data)
            if by_pct:
                total = len(new_df.loc[new_df[col_2] == j])
                data = data / total
                pct_y.append(data)
            tup = (i, j, data)
            sort_tups.append(tup)
        if by_pct:
            figures.append(go.Bar(
                name=i,
                x=x,
                y=pct_y,
                customdata=raw_y,
                text=[ f'{col_2}'.capitalize() for s in range(len(raw_y)) ],
                hovertemplate=
                f'{col_1.capitalize()}: {i}<br>' +
                '%{text}: %{x}<br>' +
                'Reps Total: %{customdata}<br>' +
                'Reps Pct: %{y:.1%}'
            ))
        else:
            figures.append(go.Bar(name=i, x=x, y=raw_y))
        
    fig = go.Figure(figures)
    fig.update_layout(barmode='stack')
    if sort_by:
        sort_tups = sorted([ tup for tup in sort_tups if tup[0] == sort_by ], key=lambda x: x[2], reverse=True)
        fig.update_layout(xaxis={'categoryorder':'array', 'categoryarray': [ val[1] for val in sort_tups ]})
    return fig

In [189]:
plot_stack(df, 'gender', 'current_party', sort_by='F', by_pct=True)

In [27]:
def vs_figure(df, column_1, column_2, in_office=True):
    current = df.loc[df['in_office'] == in_office]
    x1 = current[column_1].value_counts().index
    y1 = current[column_1].value_counts()
    x2 = current[column_2].value_counts().index
    y2 = current[column_2].value_counts()
    fig = go.Figure([
        go.Bar(x=x1, y=y1),
        go.Bar(x=x2, y=y2)
    ])

    return fig

In [29]:
fig_by_column(df, 'current_party', 'gender')

In [23]:
current['current_party'].value_counts().index

Index(['D', 'R', 'I'], dtype='object')